In [1]:
#include "kalman.hpp"

In [2]:
#include <iostream>
#include <stdexcept>
#include <vector>


In [3]:
// helper function: matrix addition
std::vector<std::vector<double>> matadd(const std::vector<std::vector<double>>& a, const std::vector<std::vector<double>>& b) {
    int rows = a.size();
    int cols = a[0].size();
    
    std::vector<std::vector<double>> result(rows, std::vector<double>(cols));

    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            result[i][j] = a[i][j] + b[i][j];
        }
    }

    return result;
}

In [4]:
// helper function: matrix subtraction
std::vector<std::vector<double>> matsub(const std::vector<std::vector<double>>& a, const std::vector<std::vector<double>>& b) {
    int rows = a.size();
    int cols = a[0].size();
    
    std::vector<std::vector<double>> result(rows, std::vector<double>(cols));

    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            result[i][j] = a[i][j] - b[i][j];
        }
    }

    return result;
}

In [5]:
// helper function: matrix transpose
std::vector<std::vector<double>> mattranspose(const std::vector<std::vector<double>>& a) {
    int rows = a.size();
    int cols = a[0].size();
    
    std::vector<std::vector<double>> result(cols, std::vector<double>(rows));

    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            result[j][i] = a[i][j];
        }
    }

    return result;
}


In [6]:
// helper function: matrix inversion
std::vector<std::vector<double>> matinverse(const std::vector<std::vector<double>>& a) {
    size_t n = a.size();
    
    if (n != a[0].size()) {
        std::cout<<" Shape of a : "<<a.size()<<","<<a[0].size()<<"\n";
        throw std::runtime_error("Only square matrices are supported for inversion.");
    }

    // Handle 1x1 matrix
    if (n == 1) {
        if (a[0][0] == 0) {
            throw std::runtime_error("Matrix is singular and cannot be inverted.");
        }
        return {{1.0 / a[0][0]}};
    }
    
    if (n == 2) {
        double determinant = a[0][0] * a[1][1] - a[0][1] * a[1][0];
        if (determinant == 0) {
            throw std::runtime_error("Matrix is singular and cannot be inverted.");
        }

        std::vector<std::vector<double>> result(2, std::vector<double>(2));
        result[0][0] = a[1][1] / determinant;
        result[0][1] = -a[0][1] / determinant;
        result[1][0] = -a[1][0] / determinant;
        result[1][1] = a[0][0] / determinant;

        return result;
    }

    if (n == 3) {
        double determinant = a[0][0]*(a[1][1]*a[2][2]-a[2][1]*a[1][2]) 
                             - a[0][1]*(a[1][0]*a[2][2]-a[1][2]*a[2][0]) 
                             + a[0][2]*(a[1][0]*a[2][1]-a[1][1]*a[2][0]);
        
        if (determinant == 0) {
            throw std::runtime_error("Matrix is singular and cannot be inverted.");
        }

        std::vector<std::vector<double>> result(3, std::vector<double>(3));

        result[0][0] = (a[1][1] * a[2][2] - a[2][1] * a[1][2]) / determinant;
        result[0][1] = (a[0][2] * a[2][1] - a[0][1] * a[2][2]) / determinant;
        result[0][2] = (a[0][1] * a[1][2] - a[0][2] * a[1][1]) / determinant;
        result[1][0] = (a[1][2] * a[2][0] - a[1][0] * a[2][2]) / determinant;
        result[1][1] = (a[0][0] * a[2][2] - a[0][2] * a[2][0]) / determinant;
        result[1][2] = (a[1][0] * a[0][2] - a[0][0] * a[1][2]) / determinant;
        result[2][0] = (a[1][0] * a[2][1] - a[2][0] * a[1][1]) / determinant;
        result[2][1] = (a[2][0] * a[0][1] - a[0][0] * a[2][1]) / determinant;
        result[2][2] = (a[0][0] * a[1][1] - a[1][0] * a[0][1]) / determinant;

        return result;
    }

    throw std::runtime_error("Only 2x2 and 3x3 matrices supported for inversion in this function.");
}



In [7]:
// helper function: matrix-vector subtraction
std::vector<std::vector<double>> matvecsub(const std::vector<std::vector<double>>& mat, const std::vector<double>& vec) {
    std::vector<std::vector<double>> result(mat.size(), std::vector<double>(vec.size()));

    for (size_t i = 0; i < mat.size(); i++) {
        for (size_t j = 0; j < vec.size(); j++) {
            result[i][j] = mat[i][j] - vec[j];
        }
    }

    return result;
}

In [8]:
// helper function: matrix-vector multiplication
std::vector<double> matvecmul(const std::vector<std::vector<double>>& a, const std::vector<double>& b) {
    int rows = a.size();
    int cols = a[0].size();

    if (cols != b.size()) {
        throw std::runtime_error("Matrix dimensions do not match for multiplication.");
    }

    std::vector<double> result(rows, 0.0);

    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            result[i] += a[i][j] * b[j];
        }
    }

    return result;
}

In [9]:
// helper function: matrix-matrix multiplication
std::vector<std::vector<double>> matmul(const std::vector<std::vector<double>>& a, const std::vector<std::vector<double>>& b) {
    int rowsA = a.size();
    int colsA = a[0].size();
    int rowsB = b.size();
    int colsB = b[0].size();

    if (colsA != rowsB) {
        throw std::runtime_error("Matrix dimensions do not match for multiplication.");
    }

    std::vector<std::vector<double>> result(rowsA, std::vector<double>(colsB, 0));

    for (int i = 0; i < rowsA; i++) {
        for (int j = 0; j < colsB; j++) {
            for (int k = 0; k < colsA; k++) {
                result[i][j] += a[i][k] * b[k][j];
            }
        }
    }

    return result;
}

In [10]:
// helper function: vector subtraction
std::vector<double> vecsub(const std::vector<double>& a, const std::vector<double>& b) {
    size_t len = a.size();
    std::vector<double> result(len);
    for (size_t i = 0; i < len; i++) {
        result[i] = a[i] - b[i];
    }
    return result;
}

In [11]:
//set up class and constructor
KalmanFilter::KalmanFilter(
    double dt,
    const std::vector<std::vector<double>>& A,
    const std::vector<std::vector<double>>& C,
    const std::vector<std::vector<double>>& Q,
    const std::vector<std::vector<double>>& R,
    const std::vector<std::vector<double>>& P)
  : A(A), C(C), Q(Q), R(R), P0(P),
    m(C.size()), n(A.size()), dt(dt), initialized(false),
    I(n, std::vector<double>(n)), x_hat(n), x_hat_new(n)
{
    for (int i = 0; i < n; i++) {
        I[i][i] = 1.0;
    }
}

KalmanFilter::KalmanFilter() {}

In [12]:
// init the kf states + measurements 
void KalmanFilter::init(double t0, const std::vector<double>& x0) {
    x_hat = x0;
    P = P0;
    this->t0 = t0;
    t = t0;
    initialized = true;
}

In [13]:
void KalmanFilter::init() {
    std::fill(x_hat.begin(), x_hat.end(), 0.0);
    P = P0;
    t0 = 0;
    t = t0;
    initialized = true;
}

In [14]:


void KalmanFilter::update(const std::vector<double>& y) {
    if (!initialized)
        throw std::runtime_error("Filter is not initialized!");

    x_hat_new = matvecmul(A, x_hat);

    P = matadd(matmul(matmul(A, P), mattranspose(A)), Q);

    std::vector<std::vector<double>> inv = matinverse(matadd(matmul(matmul(C, P), mattranspose(C)), R));

    K = matmul(matmul(P, mattranspose(C)), inv);
    std::vector<double> temp = matvecmul(C, x_hat_new);
    std::vector<double> difference = vecsub(y, temp);

    for (size_t i = 0; i < x_hat_new.size(); i++) {
        x_hat_new[i] += matvecmul(K, difference)[i];
    }

    P = matmul(matsub(I, matmul(K, C)), P);

    x_hat = x_hat_new;
    t += dt;
}


In [15]:
void KalmanFilter::update(const std::vector<double>& y, double dt, const std::vector<std::vector<double>>& A) {
    this->A = A;
    this->dt = dt;
    update(y);
}


In [16]:
std::vector<double> generateExpandedData(const std::vector<double>& baseData, int repetitions) {
    std::vector<double> expandedData;
    for (int r = 0; r < repetitions; r++) {
        for (auto val : baseData) {
            // Add slight random perturbation to the data.
            double perturbedValue = val + ((double)rand() / RAND_MAX - 0.5);
            expandedData.push_back(perturbedValue);
        }
    }
    return expandedData;
}

In [17]:
std::vector<std::vector<double>> run_kf(int expansion_factor, bool verbose) {
    
    int n = 3; // Number of states
    int m = 1; // Number of measurements

    double dt = 1.0 / 40; // Time step
    
    std::vector<double> g_preds;
    std::vector<double> g_est;
    
    std::vector<std::vector<double>> A(n, std::vector<double>(n));
    std::vector<std::vector<double>> C(m, std::vector<double>(n));
    std::vector<std::vector<double>> Q(n, std::vector<double>(n));
    std::vector<std::vector<double>> R(m, std::vector<double>(m));
    std::vector<std::vector<double>> P(n, std::vector<double>(n));
    
    A = {{1, dt, 0}, {0, 1, dt}, {0, 0, 1}};
    C = {{1, 0, 0}};
    Q = {{.05, .05, .0}, {.05, .05, .0}, {.0, .0, .0}};
    R = {{5}};
    P = {{.1, .1, .1}, {.1, 10000, 10}, {.1, 10, 100}};
    
    KalmanFilter kf(dt, A, C, Q, R, P);
    
    std::vector<double> measurements = {
      1.04202710058, 1.10726790452, 1.2913511148, 1.48485250951, 1.72825901034,
      1.74216489744, 2.11672039768, 2.14529225112, 2.16029641405, 2.21269371128,
      2.57709350237, 2.6682215744, 2.51641839428, 2.76034056782, 2.88131780617,
      2.88373786518, 2.9448468727, 2.82866600131, 3.0006601946, 3.12920591669,
      2.858361783, 2.83808170354, 2.68975330958, 2.66533185589, 2.81613499531,
      2.81003612051, 2.88321849354, 2.69789264832, 2.4342229249, 2.23464791825,
      2.30278776224, 2.02069770395, 1.94393985809, 1.82498398739, 1.52526230354,
      1.86967808173, 1.18073207847, 1.10729605087, 0.916168349913, 0.678547664519,
      0.562381751596, 0.355468474885, -0.155607486619, -0.287198661013, -0.602973173813,
      -0.873817307503, -1.144661441193, -1.415505574883, -1.686349708573, -1.957193842263, 
    -2.228037975953, -2.498882109643, -2.769726243333, -3.040570377023, -3.311414510713, 
    -3.582258644403, -3.853102778093, -4.123946911783, -4.394791045473, -4.665635179163, 
    -4.936479312853, -5.207323446543, -5.478167580233, -5.749011713923, -6.019855847613,
    -6.290699981303, -6.561544114993, -6.832388248683, -7.103232382373, -7.374076516063,
    -7.644920649753, -7.915764783443, -8.186608917133, -8.457453050823, -8.728297184513, 
    -8.999141318203, -9.269985451893, -9.540829585583, -9.811673719273, -10.082517852963,
    -10.353361986653, -10.624206120343, -10.895050254033, -11.165894387723, -11.436738521413,
    -11.707582655103, -11.978426788793, -12.249270922483, -12.520115056173
      };
    
    // std::vector<double> expandedMeasurements = generateExpandedData(measurements, expansion_factor);
    
    std::vector<double> x0 = {measurements[0], 0, 0};
    kf.init(0, x0);

    // Feed measurements into filter, output estimated states
    std::vector<double> y(m);
    if(verbose) {
        std::cout << "t = " << 0 << ", " << "x_hat[0]: ";
        for (auto& val : kf.state()){
            std::cout << val << " ";
        }
        g_est.push_back(kf.state()[2]);
        
        std::cout << std::endl;
    }
    
    int i;
    for (i = 0; i < measurements.size(); i++) {
        y[0] = measurements[i];
        kf.update(y);
        if(verbose) {
            std::cout << "t = " << (i + 1) * dt << ", y[" << i << "] = " << y[0] << ", x_hat[" << i << "] = ";
            for (auto& val : kf.state()) {
                std::cout << val << " ";
            }
            g_preds.push_back(kf.state()[2]);
            std::cout << std::endl;
        }
    }
    std::cout << std::endl;
    std::cout<<"Exec Success, Final kf states:";
    for (auto& val : kf.state()) std::cout << val << " ";
    std::cout << std::endl;
    
    std::vector<std::vector<double>> g_res;
    for (size_t i = 0; i < g_preds.size(); ++i) {
        std::vector<double> pair = {g_preds[i], g_est[i]};
        g_res.push_back(pair);
    }
    
    return g_res;
}

In [18]:
#include <chrono>
auto start = std::chrono::high_resolution_clock::now();
auto stop = std::chrono::high_resolution_clock::now();
auto duration = std::chrono::duration_cast<std::chrono::milliseconds>(stop - start);

In [19]:
std::vector<std::vector<double>> pyrun_sim(int exp_factor, bool verbose) {
    start = std::chrono::high_resolution_clock::now();
    std::vector<std::vector<double>> g_res = run_kf(exp_factor, verbose);
    stop = std::chrono::high_resolution_clock::now();
    duration = std::chrono::duration_cast<std::chrono::milliseconds>(stop - start);
    return g_res;
}

In [20]:
std::vector<std::vector<double>> g_res = pyrun_sim(1, true);

t = 0, x_hat[0]: 1.04203 0 0 
t = 0.025, y[0] = 1.04203, x_hat[0] = 1.04203 0 0 
t = 0.05, y[1] = 1.10727, x_hat[1] = 1.08709 0.898405 0.00110609 
t = 0.075, y[2] = 1.29135, x_hat[2] = 1.22062 2.38572 0.00356059 
t = 0.1, y[3] = 1.48485, x_hat[3] = 1.3876 3.46905 0.00712492 
t = 0.125, y[4] = 1.72826, x_hat[4] = 1.58997 4.40683 0.013793 
t = 0.15, y[5] = 1.74216, x_hat[5] = 1.71702 4.52169 0.0154301 
t = 0.175, y[6] = 2.11672, x_hat[6] = 1.93313 5.12414 0.031192 
t = 0.2, y[7] = 2.14529, x_hat[7] = 2.08865 5.26575 0.0374157 
t = 0.225, y[8] = 2.1603, x_hat[8] = 2.20235 5.18421 0.0316618 
t = 0.25, y[9] = 2.21269, x_hat[9] = 2.29893 5.04726 0.0173069 
t = 0.275, y[10] = 2.57709, x_hat[10] = 2.46442 5.19829 0.039685 
t = 0.3, y[11] = 2.66822, x_hat[11] = 2.61234 5.26324 0.0527055 
t = 0.325, y[12] = 2.51642, x_hat[12] = 2.69148 5.08935 0.00552738 
t = 0.35, y[13] = 2.76034, x_hat[13] = 2.80588 5.04887 -0.00849014 
t = 0.375, y[14] = 2.88132, x_hat[14] = 2.92141 5.01625 -0.0224249 
t = 0.

In [21]:
std::vector<double> py_g_pred;
std::vector<double> py_g_est;

In [22]:
int k = g_res.size();

In [23]:
std::vector<double> ret_1d_vector(std::vector<std::vector<double>> res, int axis) {
    std::vector<double> ret;
    for (int i = 0; i < res.size(); i++) {
        ret.push_back(res[i][axis]);
    }
    return ret;
}
        

In [24]:
py_g_pred = ret_1d_vector(g_res, 0);
py_g_est = ret_1d_vector(g_res, 1);

In [25]:
void printMatrix(const std::vector<double>& vec) {
    for (size_t i = 0; i < vec.size(); i++) {
            std::cout << vec[i] << " ";
        }
        std::cout << std::endl;
    }


In [26]:
printMatrix(py_g_pred);

0 0.00110609 0.00356059 0.00712492 0.013793 0.0154301 0.031192 0.0374157 0.0316618 0.0173069 0.039685 0.0527055 0.00552738 -0.00849014 -0.0224249 -0.0729322 -0.139272 -0.288577 -0.396929 -0.484796 -0.747944 -1.04221 -1.43333 -1.83638 -2.13797 -2.44388 -2.69673 -3.07251 -3.6016 -4.20755 -4.67894 -5.27548 -5.81837 -6.3351 -6.94502 -7.17158 -7.78709 -8.28638 -8.75833 -9.22649 -9.60338 -9.95378 -10.4545 -10.8401 -11.2291 -11.5854 -11.9065 -12.1908 -12.4381 -12.6488 -12.8241 -12.9653 -13.0743 -13.1531 -13.2039 -13.2289 -13.2301 -13.2098 -13.17 -13.1126 -13.0395 -12.9525 -12.853 -12.7428 -12.623 -12.4951 -12.3601 -12.2191 -12.0731 -11.9229 -11.7694 -11.6132 -11.4551 -11.2954 -11.1349 -10.9739 -10.8128 -10.652 -10.4918 -10.3325 -10.1743 -10.0175 -9.86215 -9.70853 -9.55674 -9.4069 -9.25909 -9.1134 -8.96991 


In [27]:
// %%python

// expand_factor = [1, 2, 3, 4, 5]
// benchmarks = []

// for i in expand_factor:
//     time_kf = cppyy.gbl.pyrun_sim(int(i), 0)
//     benchmarks.append(time_kf)
// 

In [28]:
%%python

import matplotlib.pyplot as plt

In [29]:
%%python

true_val = 9.81
g_pred = list(cppyy.gbl.py_g_pred)
g_pred = list(-x for x in g_pred)
x = range(len(g_pred))


# Plot the constant green line
plt.axhline(y=true_val, color='green', linestyle='-')

# Plot g_pred in orange
plt.plot(x, g_pred, color='orange', marker='o', label='KF Estimates')
plt.annotate(f'{true_val}', xy=(-0.5, true_val), color='green',
             verticalalignment='center', horizontalalignment = 'left')
# Add labels and title
plt.xlabel('Index')
plt.ylabel('Acceleration (m/s₂)')
plt.title('True Value vs. g_pred')
plt.legend()
plt.savefig("kf_plot2.jpg")
    
plt.yscale('symlog')
# Show the plot
plt.show()


<img src="1D_kf_plot.jpg">